In [5]:
import dataiku
import dataikuapi
from dataiku import pandasutils as pdu
import pandas as pd

In [3]:
design_host = dataiku.api_client().host 
print(design_host)

http://127.0.0.1:20001


In [6]:
design_client = dataikuapi.DSSClient(design_host, "FJk7Jffd8TStmT4Zvx9efC6G5mYZusec") 

In [7]:
project = design_client.get_project("DSPIPELINEFRAUD")

In [8]:
project.project_key

'DSPIPELINEFRAUD'